# Getting Started - Vertex AI GenAI Python Client

Original source: [vertex-ai-mlops/Applied GenAI/Getting Started - Vertex AI GenAI Python Client.ipynb at main · statmike/vertex-ai-mlops](https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Getting%20Started%20-%20Vertex%20AI%20GenAI%20Python%20Client.ipynb)

How to get information into (prompts) and out of (responses) of Vertex AI hosted Generative AI Models!

Vertex AI has access to hosted [generative AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview) models, called foundation models. These can be easily interacted with in the console using [Vertex AI Generative AI Studio](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/generative-ai-studio). This notebook covers interacting with the models with the [`vertexai`](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai) Python package that is include with the Vertex AI SDK [aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest).

*   Gemini API (Multimodal data, text, code, chat)
    *   `vertexai.generative_models`
*   PaLM API (Text, chat, embeddings)
    *   `vertexai.language_models`
*   Codey APIs (Code generation, code chat, code completion)
    *   `vertexai.language_models`
*   Imagen API (Image generation, image editing, image captioning, visual question answering, and multimodal embedding)
    *   `vertexai.vision_models`

The current models in GA and in preview can be reviewed in the [documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models) as well as within the [Vertex AI Model Garden](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/model-garden) which also include access to serving and tuning OSS models.

Model have names and versions. The [names](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#model_naming_scheme) are designed show both the purpose and size of the model. For instance, sizes are conveyed with names like `Bison` and `Gecko` while purposes are `text`, `textembedding`, `chat`, `code`, `imagetext`, `imagegeneration` and more. [Versions](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/model-versioning) are either `stable` and available for six months after the release of subsequent version, or `latest` which are incremenatally updated and improved but can give different responses to the same prompt.


* * *


## Colab Setup

To run this notebook in Colab click <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/model_monitoring.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a> and run the cells in this section. Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [2]:
PROJECT_ID = 'your-project-id' # replace with project ID

In [ ]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
print(PROJECT_ID)

demos-vertex-ai


## Install packages

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.storage', 'google-cloud-storage')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

In [4]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [5]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'demos-vertex-ai'

In [6]:
REGION = 'us-central1'
SERIES = 'generative-ai'
EXPERIMENT = 'getting-started'

# Set the BUCKET name for saving work:
BUCKET = PROJECT_ID

### Packages

In [7]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from google.cloud import aiplatform
from google.cloud import storage

import vertexai.vision_models # Imagen Models
import vertexai.preview.vision_models
import vertexai.language_models # PaLM and Codey Models
import vertexai.generative_models # for Gemini Models

import json
import requests
import IPython

In [8]:
aiplatform.__version__

'1.44.0'

### Clients

In [9]:
vertexai.init(project = PROJECT_ID, location = REGION)
gcs = storage.Client(project = PROJECT_ID)

bucket = gcs.lookup_bucket(BUCKET)

* * *

## Vertex AI Package

With the [vertexai](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai) client there are packages for the types of data being interacted with. There is also a higher package for preview models (not yet in GA).

> **NOTE:** In can be helpful to review the API Documentation at it's source in GitHub for up to the moment release information: [github/googleapis/python-aiplatform](https://github.com/googleapis/python-aiplatform/tree/main)

Gemini Text and Multimodal Models:

*   [vertexai.generative\_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.generative_models)
    *   [vertexai.preview.generative\_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.generative_models)

Language Models (PaLM and Codey Models):

*   [vertexai.language\_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models)
    *   [vertexai.preview.language\_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.language_models)

Vision Models (Imagen Models):

*   [vertexai.vision\_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.vision_models)
    *   [vertexai.preview.vision\_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.vision_models)

In [10]:
# Gemini Models
gemini_text = vertexai.generative_models.GenerativeModel("gemini-1.0-pro")
gemini_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-vision")

# PaLM and Codey Models
text_model = vertexai.language_models.TextGenerationModel.from_pretrained('text-bison')
chat_model = vertexai.language_models.ChatModel.from_pretrained('chat-bison')
textembed_model = vertexai.language_models.TextEmbeddingModel.from_pretrained('textembedding-gecko')
codegen_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-bison')
codecomp_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-gecko')
codechat_model = vertexai.language_models.CodeChatModel.from_pretrained('codechat-bison')

# Imagen Models
imagecap_model = vertexai.vision_models.ImageCaptioningModel.from_pretrained("imagetext")
imageqna_model = vertexai.vision_models.ImageQnAModel.from_pretrained("imagetext")
imagetext_model = vertexai.vision_models.ImageTextModel.from_pretrained("imagetext")
multimodalembed_model = vertexai.vision_models.MultiModalEmbeddingModel.from_pretrained('multimodalembedding')
imagen1_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@002')
imagen2_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@005')

In [11]:
prompt = "What are the starting points for AI?"

In [12]:
response = gemini_text.generate_content(prompt)
response

candidates {
  content {
    role: "model"
    parts {
      text: "* **Formal logic:** The study of reasoning and argumentation, and the development of formal systems to represent and reason about knowledge.\n* **Cybernetics:** The study of the control and communication systems of animals and machines, and the application of these principles to the design of intelligent systems.\n* **Information theory:** The study of the quantification, storage, and transmission of information, and the development of mathematical tools for analyzing and designing communication systems.\n* **Game theory:** The study of strategic decision-making in situations where multiple agents interact, and the development of mathematical models for predicting the outcomes of such games.\n* **Operations research:** The study of the application of mathematical methods to solve problems in a wide range of fields, including logistics, manufacturing, and finance."
    }
  }
  finish_reason: STOP
  safety_ratings {
    